In [14]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [3]:
batch_size = 64
num_classes = 10
learning_rate = 0.01
num_epochs = 20

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
all_transform = transforms.Compose([transforms.Resize((32,32)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.4914,0.4822,0.4465],
                                                         std=[0.2023,0.1994,0.2010])
                                    ])

In [9]:
train_dataset = datasets.CIFAR10(root='./data',
                                 train = True,
                                 transform = all_transform,
                                 download = True)

test_dataset = datasets.CIFAR10(root = './data',
                                train = False,
                                transform = all_transform,
                                download = True)

Files already downloaded and verified
Files already downloaded and verified


In [16]:
train_loader = DataLoader(dataset = train_dataset,
                          batch_size = batch_size,
                          shuffle = True)

test_loader = DataLoader(dataset = test_dataset,
                         batch_size = batch_size,
                         shuffle = True)

In [18]:
class ConvNeuralNet(nn.Module):
  def __init__(self, num_classes):
    super(ConvNeuralNet, self).__init__()
    self.conv_l1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
    self.conv_l2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
    self.max_pool1 = nn.MaxPool2d(kernel_size=2,stride=2)

    self.conv_l3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
    self.conv_l4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
    self.max_pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
    self.fc1 = nn.Linear(1600,128)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(128,num_classes)

  def forward(self, x):
    out = self.conv_l1(x)
    out = self.conv_l2(out)
    out = self.max_pool1(out)

    out = self.conv_l3(out)
    out = self.conv_l4(out)
    out = self.max_pool2(out)

    out = out.reshape(out.size(0), -1)

    out = self.fc1(out)
    out = self.relu1(out)
    out = self.fc2(out)
    return out

In [19]:
model = ConvNeuralNet(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, weight_decay = 0.005, momentum = 0.9)
total_step = len(train_loader)